무식한 신경망 학습
========
MNIST 이미지 인식 신경망을 역전파 없이 만들어보자. 아래와 같은 구조로 만들것임.

> 입력층(784) &rarr; 은닉층(50 or 100) &rarr; 시그모이드 &rarr; 결과(10) &rarr; 소프트맥스

In [5]:
import mnist
import numpy as np

#
# Hyper parameters
#
# 히든레이어 뉴런 수 (ex: 50, 100)
HIDDEN_LAYER_SIZE = 50
# 정규분포 난수로 생성될 초기 가중치의 표준편차
WEIGHT_INIT_STD = 0.01
# 경사하강법을 몇번 적용할지
ITERATION_COUNT = 10000
# 학습에 사용할 미니배치의 크기
BATCH_SIZE = 100
# 학습률
LEARNING_RATE = 0.1
# 에퍼크, 학습 진척도를 얼마나 자주 표시할지 (ex: 100, 300)
EPOCH = 300

#
# Utility functions
#
def sigmoid(x):
    return 1/(1 + np.exp(-x))
def gradient_sigmoid(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

def softmax(A):
    extend = (lambda x:x) if A.ndim == 1 else (lambda x: x[..., np.newaxis])
    ExpA = np.exp(A - extend(A.max(axis=-1)))
    return ExpA / extend(ExpA.sum(axis=-1))

def cross_entropy_error(expected, actual):
    epsilon = 1E-7
    return -(actual * np.log(expected + epsilon)).sum(axis=-1)
def cross_entropy_error_batch(*args):
    return cross_entropy_error(*args).mean()

def predict(input):
    def network(w0, b0, w1, b1):
        a0 = input @ w0 + b0
        z0 = sigmoid(a0)
        a1 = z0 @ w1 + b1
        z1 = softmax(a1)
        return [a0, z0, a1, z1]
    return network

def accuracy(expected, actual):
    return (expected.argmax(axis=-1) == actual.argmax(axis=-1)).mean()

#
# Main logic
#
MNIST = mnist.load()
TRAIN_IMG = MNIST['train_img']
TRAIN_LABEL = MNIST['train_label']

layer0_size = TRAIN_IMG.shape[-1]
layer1_size = HIDDEN_LAYER_SIZE
layer2_size = TRAIN_LABEL.shape[-1]

# Randomly initialize the parameters
parameters = [
    # w0
    WEIGHT_INIT_STD * np.random.randn(layer0_size, layer1_size), 
    # b0
    np.zeros(layer1_size),
    # w1
    WEIGHT_INIT_STD * np.random.randn(layer1_size, layer2_size),
    # b1
    np.zeros(layer2_size),
]

print('''학습 시작!

반복횟수\t정확도\tLoss
-------------------------------------------''')
for iteration in range(ITERATION_COUNT):
    # Sample a batch from the train image/label set
    sample = np.random.choice(TRAIN_IMG.shape[0], BATCH_SIZE)
    BATCH_IMG = TRAIN_IMG[sample]
    BATCH_LABEL = TRAIN_LABEL[sample]

    network = predict(BATCH_IMG)

    # Try the result
    if iteration % EPOCH == 0:
        expected = network(*parameters)[-1]
        percentage = accuracy(expected, BATCH_LABEL)*100
        loss = cross_entropy_error_batch(expected, BATCH_LABEL)
        print(f'{iteration:8}\t{percentage:.04}%\t{loss}')

    # Calculate gradient
    loss_function = lambda *arguments: cross_entropy_error_batch(network(*arguments)[-1], BATCH_LABEL)
    def grad_naive(parameters, h=1E-4):
        def grad(param):
            shape = param.shape
            gradient = np.empty(shape)
            for j in np.ndindex(shape):
                orig = param[j]
                param[j] = orig + h
                y2 = loss_function(*parameters)
                param[j] = orig - h
                y1 = loss_function(*parameters)
                param[j] = orig
                gradient[j] = (y2 - y1)/(2*h)
            return gradient
        #return [np.zeros(parameters[0].shape), *(grad(param) for param in parameters[1:])]
        return [grad(param) for param in parameters]
    def grad_analytic(parameters):
        w1 = parameters[2]
        a0, z0, _, expected = network(*parameters)
        
        # Backward propagation
        dz1 = (expected - BATCH_LABEL)/BATCH_SIZE
        dw1 = z0.T @ dz1
        db1 = dz1.sum(axis=0)
        
        dz0 = gradient_sigmoid(a0) * (dz1 @ w1.T)
        dw0 = BATCH_IMG.T @ dz0
        db0 = dz0.sum(axis=0)
        return [dw0, db0, dw1, db1]

    # Update parameters using gradient descent method
    gradient = grad_analytic(parameters)
    for param, grad in zip(parameters, gradient):
        param -= LEARNING_RATE * grad

expected = predict(MNIST['test_img'])(*parameters)[-1]
TEST_LABEL = MNIST['test_label']
percentage = accuracy(expected, TEST_LABEL)*100

print(f'''
학습 완료!

최종 점수
-------------
정확도 : {percentage}%
''')

학습 시작!

반복횟수	정확도	Loss
-------------------------------------------
       0	11.0%	2.3057093326098355
     300	56.0%	1.6630170021656108
     600	79.0%	0.8275390509215265
     900	85.0%	0.5581177557098227
    1200	92.0%	0.40288390492765574
    1500	88.0%	0.49844538135890226
    1800	89.0%	0.31785818016998824
    2100	90.0%	0.28806267233478844
    2400	88.0%	0.35375411964537357
    2700	92.0%	0.3196433350073984
    3000	90.0%	0.287587011873968
    3300	95.0%	0.2596750026306412
    3600	90.0%	0.34085480786688627
    3900	91.0%	0.2730142301537956
    4200	92.0%	0.2134652409435964
    4500	92.0%	0.28057109553390236
    4800	95.0%	0.3384401103156803
    5100	90.0%	0.3162968517330497
    5400	95.0%	0.1676589282608701
    5700	94.0%	0.17557476433573616
    6000	91.0%	0.34179091873328415
    6300	94.0%	0.2420873021031243
    6600	92.0%	0.273471817437171
    6900	96.0%	0.15991521303709477
    7200	90.0%	0.31844268157773137
    7500	95.0%	0.15652490922762363
    7800	94.0%	0.22284897309411797
    8

## TODOs
1. 수치미분으로 구한것과, 해석적으로 구한 두 그래디언트의 값이 실제로 얼마나 비슷한지 체크
2. numpy-mkl로 라이브러리 교체하고 성능향상 보기